<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llamaindexYouTubekv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://blog.gopenai.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

!pip install youtube-transcript-api
!pip install llama-index-readers-youtube-transcript
!pip install llama-index llama-index-llms-groq groq llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached n

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage,
    Settings
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq

In [3]:
import os
import getpass
credential_names = ["GROQ_API_KEY","HF_TOKEN"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GROQ_API_KEY··········
Provide your...HF_TOKEN··········


In [4]:
HF_TOKEN=os.environ["HF_TOKEN"]

In [5]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    #ytlinks=["https://www.youtube.com/watch?v=i3OYlaoj-BM"]
    ytlinks=["https://www.youtube.com/watch?v=rzD5i-6eUO0&t"]

)

In [6]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
print(f"Loaded {len(documents)} documents")
print(f"Split into {len(nodes)} nodes")
print(f"nodes [0] {nodes[0].metadata} ")

Loaded 1 documents
Split into 9 nodes
nodes [0] {'video_id': 'rzD5i-6eUO0'} 


In [8]:


Settings.llm = Groq(model="llama-3.1-8b-instant",api_key=os.environ["GROQ_API_KEY"])

Settings.embed_model = HuggingFaceEmbedding(
    #model_name="BAAI/bge-small-en-v1.5"
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, node_parser=nodes)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

In [10]:
vector_index.storage_context.persist(persist_dir="./storage_mini")
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()

In [11]:
query = "summarise the document"
resp = query_engine.query(query)
print(resp)

The document is a presentation about using a query engine router to route questions to either a summary tool or a vector tool, depending on the question. The presentation also discusses using a multi-dock agent to create vector query tools and summary query tools for multiple documents. Additionally, it mentions using a Knowledge Graph index to create a knowledge graph from a book on solar system exploration and India's contribution.


In [12]:
# prompt: print the text in a paragraph format

for doc in documents:
  print(doc.text)


hello all this is anirban good evening
today I'm trying to share my
recent learning and experience with the
neo4j graph
database and I was quite fascinated with
the results I got with a low code or no
code approach using the Llama index
library and uh I thought of sharing this
I follow it up with a uh router query
engine where I use multi mulle different
rag pipelines and allow the llm to
choose the best approach and finally I
use a agentic rag which has uh Knowledge
Graph as one of the
components and it chooses the best
approach to query a document start start
with i I'm starting with the knowledge
graph that I created from this book this
is a very excellent book
on on
solar system exploration and India's
contribution a beginner's guide by Dr TI
praim Das from ISRO director of Space
Science at ISRO which he shared ebook
some time back and I was reading this so
just thought of creating a knowledge
graph with this and uh what I was quite
amazed at is the power of llama index to
create a